In [10]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')

# Create a cursor object
cursor = conn.cursor()

# Query all the pages
cursor.execute('SELECT * FROM blocks')
data = cursor.fetchall()
print('Total blocks: ', len(data))

Total blocks:  5971


In [6]:
import pandas as pd
from fastembed.rerank.cross_encoder import TextCrossEncoder

(
    pd.DataFrame(TextCrossEncoder.list_supported_models())
    .sort_values("size_in_GB")
    .drop(columns=["sources", "model_file"])
    .reset_index(drop=True)
)

,model,size_in_GB,description,license
0,Xenova/ms-marco-MiniLM-L-6-v2,0.08,MiniLM-L-6-v2 model optimized for re-ranking t...,apache-2.0
1,Xenova/ms-marco-MiniLM-L-12-v2,0.12,MiniLM-L-12-v2 model optimized for re-ranking ...,apache-2.0
2,jinaai/jina-reranker-v1-tiny-en,0.13,Designed for blazing-fast re-ranking with 8K c...,apache-2.0
3,jinaai/jina-reranker-v1-turbo-en,0.15,Designed for blazing-fast re-ranking with 8K c...,apache-2.0
4,BAAI/bge-reranker-base,1.04,BGE reranker base model for cross-encoder re-r...,mit
5,jinaai/jina-reranker-v2-base-multilingual,1.11,A multi-lingual reranker model for cross-encod...,cc-by-nc-4.0


In [7]:
# Embed the content of the blocks
# Note: that the index remains the same so we can match the original data
data_to_embed = [block[3] for block in blocks]

In [8]:
data_to_embed[0]

'- [Investment research for everyone](https://openbb.co/) /tit'

In [16]:
from fastembed.rerank.cross_encoder import TextCrossEncoder
import torch

query = 'trade within countries'
encoder = TextCrossEncoder(model_name="Xenova/ms-marco-MiniLM-L-12-v2")
scores = list(encoder.rerank(query, data_to_embed))
top_scores = torch.topk(torch.tensor(scores), 5, dim=0)
print("Top scores:")
print(top_scores.values)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Top scores:
tensor([-3.2670, -5.6382, -6.5174, -6.6383, -7.2767])


In [18]:
import pandas as pd

top_pages = [(data[i][3], data[i][2], scores[i]) for i in top_scores.indices]
print("Top 5 pages content, path, and score:")
pd.set_option('display.max_colwidth', 100)
df = pd.DataFrame(top_pages, columns=["Content", "Path", "Score"])
df

Top 5 pages content, path, and score:


,Content,Path,Score
0,\t\t\t- Trade,/Users/freddycaceres/Research/Second brain/2024_03_14.md#15,-3.266987
1,\t\t- They enforce **economic** and **trade** sanctions based on US foreign policy,/Users/freddycaceres/Research/Second brain/2023_11_10.md#19,-5.638238
2,"\t\t- They target foreign countries and regimes, terrorists, international narcotics traffickers...",/Users/freddycaceres/Research/Second brain/2023_11_10.md#20,-6.517385
3,\t\t- Proprietary trading,/Users/freddycaceres/Research/Second brain/2024_04_25.md#22,-6.638289
4,\t- There is an interesting comparison between the Portuguese empire approach of building small ...,/Users/freddycaceres/Research/Second brain/2022_12_12.md#6,-7.276673
